## Input Parameters

In [1]:
# The default is to include points from all NLCD classes and MGRS grid zones
point_nlcd_keep_list = []
point_mgrs_keep_list = []

# This would show all NLCD classes for MGRS grid zone 12T
point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
point_mgrs_keep_list = ['10S']

# # This would show NLCD 82 points for all MGRS grid zones
# point_nlcd_keep_list = [82]
# point_mgrs_keep_list = []

# The minimum number of months in the target year
min_month_count = 3

# Overwrite the summary stats file everytime this block is run
output_txt = 'summary_stats_mgrs10s.txt'
output_f = open(output_txt, 'w')
output_f.write('MGRS: '+ ', '.join(point_mgrs_keep_list) + '\n')
output_f.write('NLCD: '+ ', '.join(map(str, point_nlcd_keep_list)) + '\n\n')
output_f.close()

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)

    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)
            
        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[], title='', print_flag=True, write_flag=True):
    """"""
    output = [title]

    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    output.append('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])

        output.append('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))

    if print_flag:
        print('\n'.join(output))
    if write_flag:
        with open(output_txt, 'a') as output_f:
            output_f.write('\n'.join(output+ ['\n']))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 239


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop one datapoint in each year',
    print_flag=True, write_flag=True
)

Randomly drop one datapoint in each year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1106   0.0836   0.0023   0.8554   0.0962   0.8535     1912
        climo_mean   0.1244   0.0941   0.0060   0.8140   0.1267   0.8151     1912
      climo_median   0.1278   0.0905   0.0056   0.8357   0.1122   0.8061     1912
     interp_clim_a   0.1016   0.0787   0.0041   0.8347   0.1114   0.8788     1912
     interp_clim_b   0.1017   0.0787   0.0042   0.8345   0.1116   0.8786     1912
     interp_clim_c   0.1019   0.0773   0.0040   0.8455   0.1042   0.8770     1912
       whit_a_0p50   0.1125   0.0854   0.0025   0.8661   0.0893   0.8488     1912
       whit_a_0p20   0.1119   0.0843   0.0026   0.8859   0.0766   0.8515     1912
       whit_a_0p10   0.1126   0.0844   0.0027   0.8958   0.0703   0.8506     1912
       whit_a_0p05   0.1138   0.0851   0.0028   0.9021   0.0663   0.8486     1912
       whit_a_0p01   0.1158   0.0865   0.0029   0.9082   

## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "growing" season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "growing" season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.0984   0.0730  -0.0131   0.8821   0.0672   0.9070     1911
        climo_mean   0.1187   0.0873   0.0009   0.8729   0.0875   0.8615     1911
      climo_median   0.1224   0.0838   0.0046   0.8946   0.0764   0.8545     1911
     interp_clim_a   0.0913   0.0689  -0.0061   0.8775   0.0774   0.9202     1911
     interp_clim_b   0.0915   0.0691  -0.0063   0.8769   0.0776   0.9199     1911
     interp_clim_c   0.0912   0.0671  -0.0043   0.8884   0.0718   0.9192     1911
       whit_a_0p50   0.1014   0.0758  -0.0118   0.8980   0.0577   0.9002     1911
       whit_a_0p20   0.0984   0.0728  -0.0102   0.9159   0.0471   0.9058     1911
       whit_a_0p10   0.0975   0.0717  -0.0094   0.9247   0.0419   0.9076     1911
       whit_a_0p05   0.0973   0.0714  -0.0088   0.9303   0.0386   0.9081     1911
       whit_a_0p01   0.0977 

## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1281   0.0982   0.0073   0.7893   0.1378   0.7493     1912
        climo_mean   0.1318   0.1012   0.0094   0.7215   0.1819   0.7326     1912
      climo_median   0.1347   0.0969   0.0059   0.7422   0.1656   0.7206     1912
     interp_clim_a   0.1161   0.0899   0.0083   0.7554   0.1598   0.7946     1912
     interp_clim_b   0.1161   0.0899   0.0084   0.7554   0.1599   0.7944     1912
     interp_clim_c   0.1161   0.0883   0.0066   0.7657   0.1517   0.7931     1912
       whit_a_0p50   0.1291   0.0987   0.0067   0.7946   0.1339   0.7459     1912
       whit_a_0p20   0.1297   0.0991   0.0051   0.8110   0.1222   0.7459     1912
       whit_a_0p10   0.1309   0.1001   0.0042   0.8193   0.1162   0.7431     1912
       whit_a_0p05   0.1322   0.1014   0.0037   0.8247   0.1123   0.7395     1912
       whit_a_0p01   0.13

## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a datapoint from the winter (Dec-Feb)',
    print_flag=True, write_flag=True
)

Randomly drop a datapoint from the winter (Dec-Feb)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1291   0.1006   0.0029   0.7637   0.1536   0.6896     1882
        climo_mean   0.1293   0.1013   0.0009   0.6768   0.2070   0.6803     1882
      climo_median   0.1323   0.0967  -0.0019   0.7018   0.1883   0.6671     1882
     interp_clim_a   0.1158   0.0913   0.0019   0.7202   0.1803   0.7445     1882
     interp_clim_b   0.1158   0.0913   0.0020   0.7206   0.1802   0.7444     1882
     interp_clim_c   0.1158   0.0897   0.0005   0.7327   0.1710   0.7440     1882
       whit_a_0p50   0.1301   0.1020   0.0041   0.7655   0.1536   0.6857     1882
       whit_a_0p20   0.1311   0.1024   0.0010   0.7819   0.1402   0.6849     1882
       whit_a_0p10   0.1328   0.1036  -0.0004   0.7894   0.1339   0.6802     1882
       whit_a_0p05   0.1347   0.1050  -0.0013   0.7939   0.1301   0.6742     1882
       whit_a_0p01   0.1378   0.1075  -0.0021 

## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint that is next to an existing missing data point',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint that is next to an existing missing data point
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1437   0.1146  -0.0061   0.7065   0.1834   0.7259      182
        climo_mean   0.1330   0.1022  -0.0083   0.7139   0.1764   0.7693      182
      climo_median   0.1351   0.0970  -0.0089   0.7261   0.1679   0.7594      182
     interp_clim_a   0.1262   0.0986  -0.0072   0.7102   0.1799   0.7980      182
     interp_clim_b   0.1265   0.0988  -0.0071   0.7082   0.1812   0.7974      182
     interp_clim_c   0.1260   0.0966  -0.0075   0.7163   0.1757   0.7973      182
       whit_a_0p50   0.1436   0.1164   0.0000   0.7450   0.1647   0.7256      182
       whit_a_0p20   0.1442   0.1166  -0.0023   0.7685   0.1471   0.7255      182
       whit_a_0p10   0.1462   0.1170  -0.0046   0.7805   0.1371   0.7202      182
       whit_a_0p05   0.1484   0.1182  -0.0068   0.7886   0.1296   0.7148      182
       whit_a_0p01

## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1303   0.0995   0.0001   0.7927   0.1333   0.7980     1912
        climo_mean   0.1237   0.0927   0.0075   0.8157   0.1259   0.8187     1912
      climo_median   0.1274   0.0895   0.0080   0.8388   0.1117   0.8088     1912
     interp_clim_a   0.1114   0.0846   0.0038   0.8042   0.1296   0.8558     1912
     interp_clim_b   0.1115   0.0847   0.0037   0.8041   0.1297   0.8554     1912
     interp_clim_c   0.1112   0.0832   0.0041   0.8157   0.1225   0.8551     1912
       whit_a_0p50   0.1312   0.0997   0.0005   0.8260   0.1123   0.7965     1912
       whit_a_0p20   0.1316   0.0999  -0.0003   0.8441   0.0999   0.7968     1912
       whit_a_0p10   0.1331   0.1009  -0.0009   0.8529   0.0937   0.7938     1912
       whit_a_0p05   0.1349   0.1022  -0.0012   0.8587   0.0896   0.7897     1912
       whit_a_0p01   0.1379   0.1046  -0.0016   0.86

## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1200   0.0899  -0.0026   0.8166   0.1165   0.8614     1911
        climo_mean   0.1239   0.0905   0.0016   0.8507   0.0985   0.8497     1911
      climo_median   0.1286   0.0872   0.0032   0.8733   0.0854   0.8396     1911
     interp_clim_a   0.1045   0.0783  -0.0005   0.8337   0.1075   0.8977     1911
     interp_clim_b   0.1048   0.0784  -0.0007   0.8332   0.1075   0.8971     1911
     interp_clim_c   0.1045   0.0766   0.0003   0.8450   0.1010   0.8961     1911
       whit_a_0p50   0.1215   0.0909  -0.0020   0.8554   0.0918   0.8556     1911
       whit_a_0p20   0.1201   0.0895  -0.0021   0.8756   0.0786   0.8592     1911
       whit_a_0p10   0.1203   0.0894  -0.0022   0.8856   0.0720   0.8592     1911
       whit_a_0p05   0.1211   0.0900  -0.0023   0.8921   0.0677   0.8578     1911
       whit_a_0p01   0.1232   0

## Randomly drop a two month gap during the non-growing season (Oct-Mar)

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1438   0.1118  -0.0004   0.7313   0.1711   0.6869     1912
        climo_mean   0.1312   0.1010   0.0097   0.7346   0.1791   0.7383     1912
      climo_median   0.1361   0.0979   0.0085   0.7566   0.1638   0.7198     1912
     interp_clim_a   0.1214   0.0949   0.0047   0.7329   0.1751   0.7776     1912
     interp_clim_b   0.1215   0.0950   0.0047   0.7328   0.1752   0.7773     1912
     interp_clim_c   0.1217   0.0934   0.0041   0.7440   0.1675   0.7752     1912
       whit_a_0p50   0.1433   0.1118  -0.0012   0.7537   0.1560   0.6917     1912
       whit_a_0p20   0.1449   0.1124  -0.0023   0.7717   0.1435   0.6892     1912
       whit_a_0p10   0.1476   0.1140  -0.0030   0.7801   0.1373   0.6816     1912
       whit_a_0p05   0.1506   0.1160  -0.0037   0.7851   0.1334   0.6726     1912
       whit_a_0p01   0.1552 

## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1497   0.1148   0.0047   0.7516   0.1650   0.7394     1912
        climo_mean   0.1229   0.0927   0.0012   0.8258   0.1136   0.8240     1912
      climo_median   0.1252   0.0885   0.0005   0.8478   0.0987   0.8184     1912
     interp_clim_a   0.1172   0.0900   0.0029   0.7887   0.1393   0.8436     1912
     interp_clim_b   0.1175   0.0903   0.0028   0.7881   0.1396   0.8428     1912
     interp_clim_c   0.1164   0.0881   0.0026   0.7997   0.1318   0.8445     1912
       whit_a_0p50   0.1469   0.1119   0.0052   0.8093   0.1282   0.7531     1912
       whit_a_0p20   0.1488   0.1129   0.0053   0.8298   0.1151   0.7507     1912
       whit_a_0p10   0.1516   0.1148   0.0052   0.8393   0.1089   0.7446     1912
       whit_a_0p05   0.1545   0.1171   0.0049   0.8448   0.1051   0.7375     1912
       whit_a_0p01   0.1590   0.1205   0.0045   0.

## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1339   0.1015   0.0130   0.7906   0.1447   0.8130     1911
        climo_mean   0.1138   0.0850   0.0038   0.8756   0.0821   0.8637     1911
      climo_median   0.1175   0.0820   0.0048   0.8987   0.0685   0.8566     1911
     interp_clim_a   0.1041   0.0803   0.0084   0.8331   0.1134   0.8901     1911
     interp_clim_b   0.1042   0.0804   0.0082   0.8326   0.1135   0.8898     1911
     interp_clim_c   0.1034   0.0786   0.0089   0.8446   0.1066   0.8904     1911
       whit_a_0p50   0.1357   0.1031   0.0095   0.8587   0.0984   0.8094     1911
       whit_a_0p20   0.1386   0.1052   0.0093   0.8806   0.0843   0.8052     1911
       whit_a_0p10   0.1420   0.1074   0.0091   0.8905   0.0780   0.7986     1911
       whit_a_0p05   0.1450   0.1096   0.0090   0.8962   0.0743   0.7924     1911
       whit_a_0p01   0.1494  

## Randomly drop a three month gap during the non-growing season (Oct-Mar)

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1655   0.1275  -0.0142   0.6814   0.1980   0.6200     1912
        climo_mean   0.1321   0.1021   0.0099   0.7394   0.1835   0.7540     1912
      climo_median   0.1361   0.0987   0.0090   0.7587   0.1697   0.7389     1912
     interp_clim_a   0.1311   0.1020  -0.0022   0.7104   0.1907   0.7593     1912
     interp_clim_b   0.1313   0.1021  -0.0022   0.7101   0.1910   0.7583     1912
     interp_clim_c   0.1311   0.1005  -0.0026   0.7201   0.1838   0.7581     1912
       whit_a_0p50   0.1618   0.1249  -0.0106   0.7112   0.1817   0.6382     1912
       whit_a_0p20   0.1633   0.1253  -0.0110   0.7320   0.1675   0.6371     1912
       whit_a_0p10   0.1662   0.1272  -0.0114   0.7427   0.1600   0.6299     1912
       whit_a_0p05   0.1694   0.1295  -0.0117   0.7495   0.1551   0.6212     1912
       whit_a_0p01   0.174

## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1619   0.1247   0.0011   0.7162   0.1838   0.6852     1912
        climo_mean   0.1253   0.0944   0.0072   0.8170   0.1250   0.8115     1912
      climo_median   0.1296   0.0910   0.0074   0.8370   0.1123   0.7998     1912
     interp_clim_a   0.1237   0.0955   0.0042   0.7666   0.1544   0.8189     1912
     interp_clim_b   0.1239   0.0956   0.0041   0.7661   0.1546   0.8183     1912
     interp_clim_c   0.1236   0.0941   0.0043   0.7766   0.1481   0.8181     1912
       whit_a_0p50   0.1604   0.1235   0.0010   0.7849   0.1394   0.7000     1912
       whit_a_0p20   0.1631   0.1251   0.0000   0.8108   0.1218   0.6975     1912
       whit_a_0p10   0.1668   0.1277  -0.0007   0.8227   0.1134   0.6900     1912
       whit_a_0p05   0.1706   0.1308  -0.0013   0.8297   0.1083   0.6812     1912
       whit_a_0p01   0.1764   0.1354  -0.0021   0.8

## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1499   0.1152   0.0259   0.7310   0.1907   0.7658     1911
        climo_mean   0.1198   0.0876   0.0046   0.8439   0.1003   0.8450     1911
      climo_median   0.1244   0.0850   0.0045   0.8652   0.0870   0.8338     1911
     interp_clim_a   0.1153   0.0884   0.0153   0.7874   0.1455   0.8657     1911
     interp_clim_b   0.1155   0.0885   0.0152   0.7871   0.1456   0.8653     1911
     interp_clim_c   0.1154   0.0872   0.0152   0.7981   0.1389   0.8633     1911
       whit_a_0p50   0.1486   0.1131   0.0143   0.8156   0.1273   0.7663     1911
       whit_a_0p20   0.1532   0.1159   0.0132   0.8357   0.1139   0.7563     1911
       whit_a_0p10   0.1576   0.1190   0.0130   0.8437   0.1087   0.7458     1911
       whit_a_0p05   0.1618   0.1221   0.0127   0.8481   0.1057   0.7357     1911
       whit_a_0p01   0.1679   

## Randomly drop a four month gap during the non-growing season (Oct-Mar)

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1864   0.1452  -0.0276   0.6647   0.1999   0.5410     1912
        climo_mean   0.1304   0.0995   0.0047   0.7571   0.1696   0.7565     1912
      climo_median   0.1328   0.0948   0.0045   0.7791   0.1544   0.7485     1912
     interp_clim_a   0.1365   0.1058  -0.0115   0.7109   0.1848   0.7356     1912
     interp_clim_b   0.1368   0.1060  -0.0115   0.7111   0.1846   0.7344     1912
     interp_clim_c   0.1353   0.1039  -0.0116   0.7219   0.1771   0.7401     1912
       whit_a_0p50   0.1818   0.1400  -0.0178   0.6985   0.1868   0.5633     1912
       whit_a_0p20   0.1845   0.1422  -0.0178   0.7179   0.1737   0.5607     1912
       whit_a_0p10   0.1876   0.1449  -0.0184   0.7286   0.1657   0.5547     1912
       whit_a_0p05   0.1909   0.1475  -0.0191   0.7358   0.1603   0.5476     1912
       whit_a_0p01   0.1957

## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1970   0.1538  -0.0003   0.6390   0.2332   0.5531     1912
        climo_mean   0.1271   0.0930   0.0084   0.8129   0.1294   0.8093     1912
      climo_median   0.1324   0.0901   0.0094   0.8300   0.1193   0.7947     1912
     interp_clim_a   0.1367   0.1066   0.0041   0.7260   0.1813   0.7828     1912
     interp_clim_b   0.1371   0.1069   0.0039   0.7256   0.1814   0.7814     1912
     interp_clim_c   0.1365   0.1052   0.0046   0.7345   0.1763   0.7823     1912
       whit_a_0p50   0.1943   0.1490  -0.0029   0.7160   0.1808   0.5829     1912
       whit_a_0p20   0.1988   0.1524  -0.0042   0.7409   0.1634   0.5777     1912
       whit_a_0p10   0.2034   0.1561  -0.0051   0.7515   0.1556   0.5687     1912
       whit_a_0p05   0.2078   0.1598  -0.0059   0.7573   0.1510   0.5588     1912
       whit_a_0p01   0.2144   0.1654  -0.0072   0.76

## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1938   0.1537   0.0631   0.6371   0.2811   0.6179     1911
        climo_mean   0.1216   0.0896   0.0040   0.8280   0.1073   0.8317     1911
      climo_median   0.1261   0.0857   0.0043   0.8516   0.0935   0.8202     1911
     interp_clim_a   0.1351   0.1057   0.0336   0.7326   0.1942   0.8127     1911
     interp_clim_b   0.1353   0.1058   0.0336   0.7321   0.1945   0.8123     1911
     interp_clim_c   0.1346   0.1041   0.0337   0.7443   0.1873   0.8121     1911
       whit_a_0p50   0.1908   0.1476   0.0404   0.7219   0.2075   0.6203     1911
       whit_a_0p20   0.1955   0.1501   0.0332   0.7475   0.1849   0.6087     1911
       whit_a_0p10   0.2002   0.1536   0.0299   0.7583   0.1751   0.5971     1911
       whit_a_0p05   0.2045   0.1568   0.0277   0.7639   0.1696   0.5860     1911
       whit_a_0p01   0.2110   0

## Randomly drop a six month gap during the non-growing season (Oct-Mar)

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2153   0.1669  -0.0658   0.6588   0.1701   0.4972     1912
        climo_mean   0.1273   0.0966   0.0078   0.7850   0.1564   0.7865     1912
      climo_median   0.1309   0.0931   0.0110   0.8066   0.1447   0.7765     1912
     interp_clim_a   0.1440   0.1108  -0.0290   0.7219   0.1633   0.7373     1912
     interp_clim_b   0.1444   0.1112  -0.0292   0.7217   0.1632   0.7359     1912
     interp_clim_c   0.1424   0.1089  -0.0274   0.7327   0.1574   0.7422     1912
       whit_a_0p50   0.2114   0.1624  -0.0440   0.7002   0.1633   0.5079     1912
       whit_a_0p20   0.2167   0.1652  -0.0372   0.7114   0.1623   0.4936     1912
       whit_a_0p10   0.2221   0.1686  -0.0347   0.7170   0.1609   0.4806     1912
       whit_a_0p05   0.2270   0.1723  -0.0336   0.7200   0.1600   0.4686     1912
       whit_a_0p01   0.2342 

### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )